In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Morrissey,Alsatian Cousin,Were you and he lovers? \r\nAnd would you say...
1,Alice Cooper,I'm Always Chasing Rainbows,"At the end of the rainbow, there's happiness ..."
2,Cliff Richard,Give A Little Bit More,Well i can see - that you weren't givin' enoug...
3,INXS,Newsreel Babies,Venetian blinds are crackling \r\nI like the ...
4,Alan Parsons Project,Psychobabble,Tell you 'bout a dream that I have every night...
5,R. Kelly,Memories,Girl love check 12 12 \r\nI am talking about ...
6,Omd,TALKING LOUD AND CLEAR,"Talking loud and clear, saying just what we fe..."
7,Metallica,Some Kind Of Monster,These are the eyes that can't see me \r\nThes...
8,HIM,Pretending,"Love is a flame that can't be tamed, \r\nAnd ..."
9,Unkle,Eye For An Eye,Even now in heaven \r\nThere were angels carr...


In [9]:
df['text'][0]

'Were you and he lovers?  \r\nAnd would you say so if you were?  \r\nOn a forecourt  \r\nOn a Friday  \r\nPassing my way  \r\nOh...  \r\n  \r\nWere you and he lovers?  \r\nAnd if you were, then say that you were!  \r\nOn a groundsheet  \r\nUnder canvas  \r\nWith your tent-flap  \r\nOpen wide  \r\n  \r\nA note upon his desk  \r\n"P.S. Bring Me Home And Have Me!"  \r\nLeather elbows on a tweed coat  \r\n-Oh!-  \r\nIs that the best you can do ?  \r\nSo came his reply :  \r\n"But on the desk is where I want you!"  \r\n  \r\nSo I ask (even though I know):  \r\nWere you and he lovers ?\r\n\r\n'

In [10]:
# df = df.sample(5000)

In [11]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [12]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [13]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [14]:
import nltk
nltk.download('punkt')

def tokenization(text):
    return nltk.word_tokenize(text)

df['text'] = df['text'].apply(lambda x: tokenization(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# Assuming 'text' column contains lists of strings
df['text'] = df['text'].apply(lambda x: ' '.join(x))


In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your DataFrame 'df' with the 'text' column

tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [18]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [19]:
similarity[0]

array([1.        , 0.        , 0.00565643, ..., 0.01203575, 0.01771784,
       0.02342359])

In [20]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [21]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [22]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your DataFrame 'df' with the 'text' column

tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

def recommendation(song_df):
    if song_df not in df['song'].values:
        return "The input song is not found in the DataFrame."
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    songs = []
    for i, distance in distances[1:6]:
        songs.append(df.iloc[i]['song'])
    return songs


In [23]:
recommendation('Us Against The World')

['Make It With You',
 'Secrets',
 'Lost',
 "Hits Medley '76",
 'The Hands That Built America']

In [24]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))